In [1]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')



In [2]:
from IPython.display import Javascript, clear_output, display
from ipywidgets import widgets, interact

def start(b):
    display(Javascript('IPython.notebook.execute_cells_below()'))

startbutton = widgets.Button(description="Start")
startbutton.on_click(start)
display(startbutton)

<IPython.core.display.Javascript object>

Button(description='Start', style=ButtonStyle())

NameError: name 'widgets' is not defined

In [ ]:
import seaborn as sbn
import matplotlib.pyplot as plt
import numpy as np
from IPython.core.pylabtools import figsize
sbn.set_context("talk")
from scipy.integrate import odeint
%matplotlib notebook

def homeostat(y, t, signal, k = .2, K_d = 0.1, n=6):
    x = signal.signal[np.searchsorted(signal.time, t)-1]
    rate = x - k*y**n/(y**n+K_d**n)-y
    return rate

def signal_limiter(y, t, signal, k=.8, n=5):
    x = signal.signal[np.searchsorted(signal.time, t)-1]
    rate = x - k*y**n-y
    return rate

# System of ODEs
def adaptive_system(Y, t, signal, k=0.05, n=5, tau=0.01):
    x = signal.signal[np.searchsorted(signal.time, t)-1]
    beta, y = Y
    rate_y = x*(1-beta)-y
    rate_beta = tau*(y-k*beta)
    return [rate_beta, rate_y]

def transient_generator(Y, t, signal, k1=0.8, k2=0.05, n=5, tau=0.01):
    x = signal.signal[np.searchsorted(signal.time, t)-1]
    beta, y = Y
    rate_y = x-y*k1*beta**n-y
    rate_beta = tau*(y-k2*beta)
    return [rate_beta, rate_y]
    
class Signal:
    def __init__(self, t):
        self.time = np.linspace(0, t, 500)
        self.signal = np.zeros_like(self.time)
        
    def Step(self, steptime, stepsize):
        self.signal += stepsize*np.heaviside(self.time-steptime, steptime)

    def Pulse(self, pulsetime, pulseduration, pulsesize):
        self.Step(pulsetime, pulsesize)
        self.Step(pulsetime+pulseduration, -pulsesize)

    def MultiStep(self, times, steps):
        for time, step in zip(times,steps): self.Step(time,step)

    def AddBasalLevel(self, basal):
        self.signal += basal
        
    #this breaks stuff later, do not use
    def AddNoise(self, freq, amp, start, end):
        t = np.heaviside(self.time-start, start)
        t -= np.heaviside(self.time-end, end)
        self.signal += amp*np.sin(t*self.time/freq)
        
    
    def Plot(self):
        
        fig = plt.figure(figsize=(6,6))
        ax = fig.add_subplot(111)
        ax.plot(self.time, self.signal)
        plt.show()
        return fig, ax


In [ ]:
def AddStep(t, s):
    signal.Step(t,s)
    
def AddPulse(t, d, s):
    signal.Pulse(t, d, s)

def AddBasal(s):
    signal.AddBasalLevel(s)
    
def AddNoise(f, a, start, end):
    signal.AddNoise(f, a, start, end)

def Reset(signal):
    signal=Signal(1000)


## Equations (Transfer functions) for different feedback motifs

Using motifs found in Brandman, O., and Meyer, T.. Feedback Loops Shape Cellular Signals in Space and Time. *Science* (2008) 322: 390-395.

We will be looking at how different types of motifs respond to different types of signals. 

### Variables:
-  $x$ = input
-  $y$ = output
-  $\beta$ = intermediate
-  $k_i$ = kinetic parameter
-  $K_d$ = dissociation constant (EC50)
-  $n$ = hill coefficient or power law 
-  $\tau_\beta$ = intermediate integration time constant

### Negative Feedback
-  __basal homeostat__: Reduce the impact of any individual change on the output

$$ \frac{dy}{dt} = x - k_1 * \frac{y^n}{y^n+K_d^n} - y $$

-  __limiter__: Prevent changes in output past a certain level

$$ \frac{dy}{dt} = x - k_1*y^n - y $$

-  __adaptive system__: Spike in output to relatively small deviations

$$ \frac{dy}{dt} = x*(1-\beta) - y$$

$$ \frac{d\beta}{dt} = \tau_{\beta}*(y-k_1*\beta)$$

-  __transient generator__: Return output to basal level after perturbation to input

$$ \frac{dy}{dt} = x - k_1*\beta^n - y$$

$$ \frac{d\beta}{dt} = \tau_{\beta}*(y-k_2*\beta)$$



## Make a signal

In [ ]:
signal = Signal(1000)
fig_signal, ax_signal = signal.Plot()

step_time = widgets.FloatText(description= r'Step Time', value=0, min=0, max=1000)
step_size = widgets.FloatText(description= r'Step Size', value=0, min=-2, max=2)
step_button = widgets.Button(description="Add Step")
display(step_time, step_size, step_button)
step_button.on_click(StepButton)


def redraw():
    ax_signal.cla()
    ax_signal.plot(signal.time, signal.signal)
    fig_signal.canvas.draw()

def StepButton(b):
    AddStep(step_time.value, step_size.value)
    redraw()


def PulseButton(b):
    AddPulse(pulse_time.value, pulse_duration.value, pulse_size.value)
    redraw()
    
def BasalButton(b):
    AddBasal()
    redraw()
    
def NoiseButton(b):
    AddNoise(noise_frequency.value, noise_amplitude.value, noise_start.value, noise_end.value)
    redraw()
    
def ResetButton(b):
    display(Javascript('IPython.notebook.execute_cell()'))

pulse_time = widgets.FloatText(description= r'Pulse Time', value=0,min=0, max=1000)
pulse_size = widgets.FloatText(description= r'Pulse Size', value=0, min=-2, max=2)
pulse_duration = widgets.FloatText(description= r'Pulse Duration', value=1,min=0, max=1000-pulse_time.value)
pulse_button = widgets.Button(description="Add Pulse")
reset_button = widgets.Button(description="Reset Signal")
display(pulse_time, pulse_size, pulse_duration, pulse_button)
pulse_button.on_click(PulseButton)

basal = widgets.FloatText(description= r'Basal Value', value=0, min=-2, max=2)
basal_button = widgets.Button(description="Add Basal Value")
display(basal, basal_button)
basal_button.on_click(BasalButton)

noise_frequency = widgets.FloatText(description= r'Noise Frequency', value=0)
noise_amplitude = widgets.FloatText(description= r'Noise Amplitude', value=0, min=0, max=2)
noise_start = widgets.FloatText(description= r'Noise Start', value=0,min=0, max=999)
noise_end = widgets.FloatText(description= r'Noise End', value=0, min=noise_start.value, max=1000)
noise_button = widgets.Button(description="Add Noise")
display(noise_frequency, noise_amplitude, noise_start, noise_end, noise_button)
noise_button.on_click(NoiseButton)

display(reset_button)
reset_button.on_click(ResetButton)

## Motif parameters

In [ ]:


Kd = widgets.FloatText(description= r'$K_d$', value=.1)
k1 = widgets.FloatText(description= r'$k_1$', value=.2)
k2 = widgets.FloatText(description= r'$k_2$', value=.05)
n = widgets.FloatText(description= r'$n$', value=6)
tau = widgets.FloatText(description= r'$\tau_{\beta}$', value=.01)


motif = widgets.ToggleButtons(
    options=['basal homeostat', 'signal limiter', 'adaptive system', 'transient generator'],
    description='Motif:',
    disabled=False)

display(Kd, k1, k2, n, tau, motif)


In [ ]:
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))

button = widgets.Button(description="Create plots")
button.on_click(run_all)
display(button)

In [ ]:
if motif.value == 'basal homeostat':
    sol = odeint(homeostat, 0, signal.time, args=(signal,k1.value, Kd.value, n.value))
elif motif.value == 'signal limiter':
    sol = odeint(signal_limiter, 0, signal.time, args=(signal,k1.value, n.value))
elif motif.value == 'adaptive system':
    sol = odeint(adaptive_system, [0,0], signal.time, args=(signal,k1.value, n.value, tau.value))
    sol = sol[:,1]
elif motif.value == 'transient generator':
    sol = odeint(transient_generator, [0,0], signal.time, args=(signal,k1.value, k2.value, n.value, tau.value))
    sol = sol[:,1]

fig, axes = plt.subplots(1, 2, figsize = (9, 5), sharey=True)
fig.tight_layout()   

axes[0].plot(signal.time, signal.signal)
axes[0].set_title('Signal')

axes[1].plot(signal.time, sol)
axes[1].set_title('Response: {}'.format(motif.value))

fig.text(0.5, .02, 'time', ha='center', va='center')
fig.text(0.06, 0.5, '', ha='center', va='center', rotation='vertical')

plt.show()